In [1]:
#import necessary libraries
import requests
import json
import tweepy
import pandas as pd
import numpy as np
import os
from datetime import datetime
import matplotlib.pyplot as plt
import sys
import time
import logging
import twitter
import urllib.parse
import objectpath
% matplotlib inline

In [2]:
talentlinks = pd.read_csv('talent_tweets.csv')
talentlinks.head()

,Twitter,Link,id
0,Kim,https://twitter.com/KimKardashian/status/10910...,1091087818997170176
1,Khloe,https://twitter.com/khloekardashian/status/108...,1089033719543787521
2,Kourtney,https://twitter.com/kourtneykardash/status/109...,1090445264895148033
3,Kris,https://twitter.com/KrisJenner/status/10816124...,1081612447692148736
4,Kendall,https://twitter.com/KendallJenner/status/10772...,1077299206631481344


In [ ]:
'''from os import environ as e

t = twitter.Api(
    consumer_key=e["IKwvp6CJRAdhlu3KWB8Sp6yg7"],
    consumer_secret=e["3xS168uN3mzr2A4wleSgJzzzipntBzp6VIMf7FE9m9Retl5XOi"],
    access_token_key=e["961779940868833281-jNTXHmwP05nAkcluus6nEfxalvjxeAT"],
    access_token_secret=e["dx6NfQDsJPAIK4jdy1WPvoqrkOIAzz2GO2u9SByA3X1Ou"],
    sleep_on_rate_limit=True
)

def tweet_url(t):
    return "https://twitter.com/%s/status/%s" % (t.user.screen_name, t.id)

def get_tweets(filename):
    for line in open(filename):
        yield twitter.Status.NewFromJsonDict(json.loads(line))

def get_replies(tweet):
    user = tweet.user.screen_name
    tweet_id = tweet.id
    max_id = None
    logging.info("looking for replies to: %s" % tweet_url(tweet))
    while True:
        q = urllib.parse.urlencode({"q": "to:%s" % user})
        try:
            replies = t.GetSearch(raw_query=q, since_id=tweet_id, max_id=max_id, count=100)
        except twitter.error.TwitterError as e:
            logging.error("caught twitter api error: %s", e)
            time.sleep(60)
            continue
        for reply in replies:
            logging.info("examining: %s" % tweet_url(reply))
            if reply.in_reply_to_status_id == tweet_id:
                logging.info("found reply: %s" % tweet_url(reply))
                yield reply
                # recursive magic to also get the replies to this reply
                for reply_to_reply in get_replies(reply):
                    yield reply_to_reply
            max_id = reply.id
        if len(replies) != 100:
            break

if __name__ == "__main__":
    logging.basicConfig(filename="replies.log", level=logging.INFO)
    tweets_file = sys.argv[1]
    for tweet in get_tweets(tweets_file):
        for reply in get_replies(tweet):
            print(reply.AsJsonString())'''

In [3]:
# Use tweet IDs in archive to query Twitter API for JSON data of: retweet count, favorite count, comment count
consumer_key = 'IKwvp6CJRAdhlu3KWB8Sp6yg7'
consumer_secret = '3xS168uN3mzr2A4wleSgJzzzipntBzp6VIMf7FE9m9Retl5XOi'
access_token = '961779940868833281-jNTXHmwP05nAkcluus6nEfxalvjxeAT'
access_token_secret = 'dx6NfQDsJPAIK4jdy1WPvoqrkOIAzz2GO2u9SByA3X1Ou'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

user = api.get_user('KimKardashian')

print (user.screen_name)
print (user.followers_count)

KimKardashian
59488514


In [4]:
#let's take a look at a tweets data dictionary to identify what else we can query
import pprint

tweet1 = str(api.get_status(talentlinks['id'][0], tweet_mode='extended'))
pp = pprint.PrettyPrinter(indent=10)
pp.pprint(tweet1)

("Status(_api=<tweepy.api.API object at 0x115b7ee10>, _json={'created_at': "
 "'Thu Jan 31 21:36:22 +0000 2019', 'id': 1091087818997170176, 'id_str': "
 "'1091087818997170176', 'full_text': 'A little behind the scenes at my KKW "
 "HEARTS shoot 💛 https://t.co/fCCVhfJxUS', 'truncated': False, "
 "'display_text_range': [0, 51], 'entities': {'hashtags': [], 'symbols': [], "
 "'user_mentions': [], 'urls': [], 'media': [{'id': 1091087762344665088, "
 "'id_str': '1091087762344665088', 'indices': [52, 75], 'media_url': "
 "'http://pbs.twimg.com/ext_tw_video_thumb/1091087762344665088/pu/img/upog7B3LNg2tp2IG.jpg', "
 "'media_url_https': "
 "'https://pbs.twimg.com/ext_tw_video_thumb/1091087762344665088/pu/img/upog7B3LNg2tp2IG.jpg', "
 "'url': 'https://t.co/fCCVhfJxUS', 'display_url': "
 "'pic.twitter.com/fCCVhfJxUS', 'expanded_url': "
 "'https://twitter.com/KimKardashian/status/1091087818997170176/video/1', "
 "'type': 'photo', 'sizes': {'thumb': {'w': 150, 'h': 150, 'resize': 'crop'}, "
 "'medi

In [12]:
# list of dictionaries of desired info from tweets to build and convert to a DataFrame
#record start time of running code 
start_time = datetime.now() 

tweet_data = []
tweet_errors = []

with open('tweet_json.txt', 'w') as file:
    for id in talentlinks['id']:
        try:
            #query each tweet for which there is a matching tweet_id in archive
            tweet = api.get_status(id, tweet_mode='extended')
            #path = objectpath.Tree(tweet._json['extended_entities'])
            screen_name = tweet._json['user']['screen_name']
            tweet_id = tweet._json['id']
            #media_type = tweet._json['entities']['extended_entities']['media']['type']
            full_text = tweet._json['full_text']
            retweet_count = tweet._json['retweet_count']
            favorite_count = tweet._json['favorite_count']
            
            #add all queried data to tweet_data
            tweet_data.append({'handle': screen_name,
                               'tweet_id': tweet_id,
                               #'media_type': media_type,
                               'full_text': full_text,
                               'retweet_count': retweet_count,
                               'favorite_count': favorite_count})
        except tweepy.TweepError: #deal with tweets that bring up errors, i.e. are no longer there
            tweet_errors.append(tweet_id)
            continue
    json.dump(tweet_data, file)

#record time elapsed for code to run
time_elapsed = datetime.now() - start_time 
print('Time elapsed (hh:mm:ss.ms) {}'.format(time_elapsed))

KeyError: 'extended_entities'

In [10]:
#with open('tweet_json.txt', 'r') as fp:
tweet_data = pd.DataFrame(tweet_data, columns = ['handle','tweet_id','full_text','retweet_count','favorite_count','is_quote_status'])
tweet_data

,handle,tweet_id,full_text,retweet_count,favorite_count,is_quote_status
0,KimKardashian,1091087818997170176,A little behind the scenes at my KKW HEARTS sh...,845,8406,NaN
1,khloekardashian,1089033719543787521,Some of you are wayyyyy too serious! Not every...,5632,42714,NaN
2,kourtneykardash,1090445264895148033,Khloé is pissing me off today. https://t.co/KL...,14325,158139,NaN
3,KrisJenner,1081612447692148736,I’m so proud of my darling @KendallJenner for ...,1626,21189,NaN
4,KendallJenner,1077299206631481344,HA well this is awkward https://t.co/gyyxVZLSDE,45393,487016,NaN
5,KylieJenner,1089811412191309824,I miss my baby so much when she she’s asleep 😢...,6379,144841,NaN


In [11]:
tweet = api.get_status(id, tweet_mode='extended')
tweet._json.keys()

dict_keys(['created_at', 'id', 'id_str', 'full_text', 'truncated', 'display_text_range', 'entities', 'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'lang'])

In [ ]:
tweet_data.to_csv('')